Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots,ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,[0,T])
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

Φ = A\eye(n);

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4x4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

2.7755575615628914e-16 - 0.2683546905319056im
3.664603343001005e-16 + 0.2683546905319056im
0.03747531973228947 + 1.0000000000000007im
-0.037475319732288966 + 0.9999999999999999im

Floquet Multipliers = 

0.6651802570066604 - 0.7466828146465907im
0.6651802570066606 + 0.7466828146465908im
-1.1249427991479377 - 2.3601090341805883e-15im
-0.8889340869219562 + 5.036290646901533e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

208

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,[0,10T])
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

[Plots.jl] Initializing backend: gr


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,149.476 35.4377,149.458 35.4471,149.422 35.4611,149.367 35.4797,149.295 35.5031,149.204 35.5311,149.094 35.5637,148.966 35.601,148.82 35.643,148.655 
 35.6897,148.471 35.741,148.269 35.797,148.047 35.8576,147.806 35.9229,147.546 35.9928,147.266 36.0674,146.966 36.1467,146.646 36.2306,146.306 36.3192,145.945 
 36.4124,145.563 36.5103,145.161 36.6128,144.737 36.72,144.291 36.8318,143.824 36.9482,143.334 37.0693,142.822 37.1951,142.287 37.3254,141.728 37.4604,141.146 
 37.6001,140.541 37.7443,139.91 37.8932,139.256 38.0467,138.576 38.2049,137.871 38.3677,137.141 38.535,136.384 38.707,135.602 38.8836,134.792 39.0649,133.956 
 39.2507,133.092 39.4411,132.2 39.6361,131.281 39.8357,130.333 40.04,129.357 40.2488,128.352 40.4621,127.319 40.6801,126.256 40.9027,125.164 41.1298,124.042 
 41.3615,122.891 41.5977,121.71 41.8386,120.5 42.084,119.26 42.3339,117.991 42.5884,116.692 42.8474,115.364 43.111,114.007 43.3791,112.622 43.6518,111.209 
 43.929,109.767 44.2107,108.299 44.4969,106.804 44.7877,105.283 45.0829,103.737 45.3827,102.167 45.687,100.574 45.9957,98.9582 46.309,97.3218 46.6267,95.6659 
 46.9489,93.9919 47.2756,92.3012 47.6067,90.5957 47.9424,88.877 48.2824,87.1472 48.627,85.4083 48.976,83.6625 49.3294,81.9121 49.6872,80.1598 50.0495,78.4081 
 50.4162,76.6599 50.7873,74.918 51.1629,73.1857 51.5428,71.4662 51.9271,69.7629 52.3159,68.0794 52.709,66.4194 53.1065,64.7869 53.5083,63.1859 53.9145,61.6207 
 54.3251,60.0956 54.7401,58.6152 55.1594,57.1842 55.583,55.8073 56.0109,54.4896 56.4432,53.2363 56.8798,52.0524 57.3207,50.9436 57.7659,49.9152 58.2154,48.9729 
 58.6692,48.1224 59.1272,47.3697 59.5896,46.7205 60.0562,46.1809 60.527,45.7571 61.0021,45.455 61.4815,45.281 61.965,45.2411 62.4528,45.3415 62.9449,45.5886 
 63.4411,45.9884 63.9415,46.547 64.4461,47.2704 64.9549,48.1647 65.4679,49.2356 65.9851,50.4887 66.5064,51.9297 67.0318,53.5636 67.5614,55.3956 68.0951,57.4305 
 68.633,59.6726 69.1749,62.1261 69.721,64.7946 70.2711,67.6815 70.8254,70.7896 71.3837,74.121 71.9461,77.6776 72.5125,81.4604 73.083,85.47 73.6575,89.706 
 74.2361,94.1675 74.8187,98.8527 75.4053,103.759 75.9959,108.883 76.5905,114.22 77.189,119.764 77.7916,125.511 78.3981,131.45 79.0085,137.576 79.6229,143.877 
 80.2412,150.343 80.8635,156.962 81.4896,163.722 82.1197,170.608 82.7536,177.604 83.3914,184.695 84.0331,191.864 84.6787,199.091 85.3281,206.357 85.9813,213.642 
 86.6384,220.925 87.2992,228.183 87.9639,235.393 88.6323,242.534 89.3046,249.579 89.9806,256.506 90.6604,263.29 91.3439,269.905 92.0311,276.328 92.7221,282.533 
 93.4168,288.497 94.1152,294.196 94.8173,299.606 95.523,304.706 96.2324,309.473 96.9455,313.887 97.6622,317.93 98.3826,321.582 99.1065,324.828 99.8341,327.652 
 100.565,330.042 101.3,331.986 102.038,333.473 102.78,334.498 103.526,335.053 104.275,335.135 105.027,334.742 105.783,333.876 106.543,332.538 107.305,330.733 
 108.072,328.467 108.842,325.75 109.615,322.592 110.392,319.004 111.172,315.002 111.956,310.602 112.743,305.82 113.533,300.676 114.327,295.192 115.124,289.388 
 115.924,283.289 116.728,276.919 117.535,270.304 118.345,263.471 119.159,256.446 119.976,249.259 120.796,241.94 121.62,234.516 122.446,227.02 123.276,219.481 
 124.109,211.931 124.945,204.4 125.785,196.919 126.627,189.521 127.473,182.235 128.322,175.091 129.174,168.12 130.029,161.351 130.887,154.812 131.749,148.53 
 132.613,142.53 133.48,136.838 134.351,131.476 135.224,126.464 136.101,121.821 136.98,117.565 137.862,113.708 138.748,110.263 139.636,107.237 140.527,104.636 
 141.422,102.463 142.319,100.718 143.219,99.3948 144.121,98.4879 145.027,97.9864 145.936,97.8768 146.847,98.1421 147.761,98.7628 148.678,99.7162 149.598,100.977 
 150.52,102.518 151.446,104.